In [2]:
# import the packages
import spacy
import re
import pandas as pd

from PyPDF2 import PdfFileReader
from nltk.tokenize import word_tokenize
from itertools import compress

from pathlib import Path
import webbrowser

sp = spacy.load('en_core_web_sm')
all_stopwords = sp.Defaults.stop_words

In [3]:
# file_path = open('./pdf/REVISED-PRESS-RELEASE-SUNDAY-19TH-JULY-2020-1.pdf','rb')
# file_path = open('./pdf/Press-Statement-on-Covid-19-10th-August-2021.pdf','rb')
# file_path = open('./pdf/Press-Statement-on-Covid-19-9th-September-2021.pdf','rb')
file_path = open('/content/Press-Statement-on-Covid-19-1st-August-2021.pdf','rb')

pdf = PdfFileReader(file_path)

In [4]:
i = 0
text = ''
while i < pdf.getNumPages():
    pageinfo = pdf.getPage(i)
    text = text + ''.join(pageinfo.extractText())
    text = text.replace("\n", "").strip() 

    i+=1


In [5]:
text_tokens = word_tokenize(text) # Tokenize the text
tokens_without_sw = [word for word in text_tokens if not word in all_stopwords] # remove stopwords
textString = ' '.join([str(elem) for elem in tokens_without_sw]) # convert the list to strings
textString  = re.sub(r'[()]?','',textString) # Clean the string data
textString = textString.replace(',','').strip() # replace commas with spaces

textString = textString.replace("Homa Bay", "HomaBay")
textString = textString.replace("West Pokot","WestPokot")
textString = textString.replace("Tharaka Nithi ","TharakaNithi ")
textString = textString.replace("Murang’a","Muranga") 
textString = textString.replace("Tana River","TanaRiver")
textString = textString.replace("Taita Taveta","TaitaTaveta")
textString = textString.replace("Trans Nzoia","TransNzoia")
textString = textString.replace("Uasin Gishu","UasinGishu")
textString = textString.replace("Elgeyo Marakwet","ElgeyoMarakwet")

In [6]:
# List of all the counties
counties = ['Mombasa', 'Kwale', 'Kilifi', 'TanaRiver', 'Lamu', 'TaitaTaveta', 'Garissa', 'Wajir', 'Mandera', 'Marsabit', 
'Isiolo', 'Meru', 'TharakaNithi', 'Embu', 'Kitui', 'Machakos', 'Makueni', 'Nyandarua', 'Nyeri', 'Kirinyaga', 'Muranga', 
'Kiambu', 'Turkana', 'WestPokot', 'Samburu', 'TransNzoia', 'UasinGishu', 'ElgeyoMarakwet', 'Nandi', 'Baringo', 'Laikipia', 
'Nakuru', 'Narok', 'Kajiado', 'Kericho', 'Bomet', 'Kakamega', 'Vihiga', 'Bungoma', 'Busia', 'Siaya', 'Kisumu', 'HomaBay', 
'Migori', 'Kisii', 'Nyamira', 'Nairobi']

In [7]:
# check for counties that had positive cases in the day
def check_strings(counties, text):
    regexp = re.compile('|'.join([re.escape(x) for x in counties]))
    found = set(regexp.findall(text))
    return [x in found for x in counties]

In [8]:
counties_in_bool = check_strings(counties, textString)
counties_in_names = list(compress(counties, counties_in_bool)) # names of counties that had positive cases on the day

In [9]:
# Find a patter of names with numbers that correspond to county name and recorded number of positive cases
day = []
pattern = re.findall('([A-Za-z]+\s*[A-Za-z]*\s*[A-Za-z]*\s*[A-Za-z]*\s*[A-Za-z]*\s*[A-Za-z]*)\s?(\d+)',textString)
for p in pattern:
    n = len(p[0].split())
    if n == 1:
        if p[0].strip() in counties_in_names:
            day.append(p)
    else:
        for i in range(n):
            if (p[0].split())[i].strip() in counties_in_names:
                day.append((p[0].split()[i],p[1]))
day = set(day)

In [10]:
pd_day = pd.DataFrame(day, columns = ['County','New Positive cases'])
pd_day['New Positive cases'] = pd.to_numeric(pd_day['New Positive cases'],errors='coerce')

In [14]:
pd_day.sort_values(by='New Positive cases',ascending=False)
pd_day.to_excel('August01.xlsx',sheet_name='August01')